In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import collections
import datetime

In [2]:
def find_author(tweet):
    resul = ""
    elements = tweet.find("span", class_="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0").contents
    for elem in elements:
        if elem.find("div"):
            if 'title' in elem.find('div').attrs:
                resul += f"[{elem.find('div').attrs['aria-label']}]"
            else:
                resul += elem.find('div').attrs['aria-label']
        else:
            resul += elem.getText()
    return resul

def find_text(tweet):
    text_ = tweet.find("div", class_="css-901oao r-jwli3a r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-bnwqim r-qvutc0")

    if not text_:
        text_ = tweet.find("div", class_="css-901oao r-jwli3a r-1tl8opc r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-bnwqim r-qvutc0")  # Japonais par exemple

    lang = text_.attrs['lang']
    text = ""
    for elem in text_.contents:
        if elem.find("div"):
            if 'title' in elem.find('div').attrs:
                text += f"[{elem.find('div').attrs['aria-label']}]"
            else:
                text += elem.find('div').attrs['aria-label']
        else:
            text += elem.getText()
    return text, lang

In [3]:
FILES = os.listdir('web')

data = []

for file in FILES:
    print(file)
    search = os.path.splitext(file)[0]
    with open(f"web/{file}", "r") as f:
        page = BeautifulSoup(f.read(), 'lxml')

        tweets = page.findAll("div", class_="css-1dbjc4n r-1iusvr4 r-16y2uox r-1777fci r-1mi0q7o")
        i = 0
        for tweet in tweets:
            url = tweet.find("a", class_="css-4rbku5 css-18t94o4 css-901oao r-111h2gw r-1loqt21 r-1q142lx r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-3s2u2q r-qvutc0").attrs.get("href")
            # Ok unique url

            error_author = False
            try:
                author = find_author(tweet)
            except:
                error_author = True
                print("Erreur auteur :", url)

            time = tweet.find("time").attrs.get("datetime")
            time = datetime.datetime.strptime(time, '%Y-%m-%dT%H:%M:%S.%fZ')
            # Parfois 2 dates lors des retweet, mais c'est bien la première date qu'il faut prendre en compte

            try:
                text, lang = find_text(tweet)
            except:
                text = ""
                print("Erreur texte", i, url)

            reply = False
            reply_ = tweet.find("div", class_="css-901oao r-111h2gw r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-qvutc0")
            if reply_:
                if reply_.find(text=True) == "Replying to ":
                    reply = True
            
            if not error_author:
                data.append({'search': search, 'author': author, 'time': time, 'url': url, 'text': text, 'lang':lang, 'reply':reply})
                
            i += 1

biden.html
Erreur auteur : /TrixxSteph/status/1300944463922163713
trump.html


In [4]:
df = pd.DataFrame(data)
len(df)

3355

In [5]:
df.dtypes

search            object
author            object
time      datetime64[ns]
url               object
text              object
lang              object
reply               bool
dtype: object

In [6]:
df[df.search == "trump"]

,search,author,time,url,text,lang,reply
1810,trump,Fabulous Abomination 🏳️‍🌈,2020-09-01 23:59:59,/FabulousHeathen/status/1300946538856939525,"Sadly, it's going to be 4 more years of Trump.",en,True
1811,trump,ᴮᴱ⁷BANGᴛᴀɴɴɪᴇ✜𖧵 | Via’s Only,2020-09-01 23:59:59,/chicky_nug1bts1/status/1300946538840109056,Love trump is fucking mediocre but Biden ain’t...,en,True
1812,trump,Zack Davisson,2020-09-01 23:59:59,/ZackDavisson/status/1300946538416406528,Nellie Hall would like to show Donald Trump EX...,en,False
1813,trump,Snipes01,2020-09-01 23:59:59,/Snipes0_1/status/1300946538265370626,No! You are not putting this one Black people ...,en,False
1814,trump,Neighbors & Citizens,2020-09-01 23:59:59,/CoalitionofNC/status/1300946538089373697,There guy is President now. They are touring T...,en,False
...,...,...,...,...,...,...,...
3350,trump,Cerdic Conan,2020-09-01 23:57:22,/CerdicConan/status/1300945877016616961,Donald Trump is the ultimate political outside...,en,False
3351,trump,NoMad,2020-09-01 23:57:22,/PDXDJnomad/status/1300945876848799745,"As a rule, I don't generally make fun of peopl...",en,False
3352,trump,"Gen Trump, Brotherly Leader & Guide of Revolution",2020-09-01 23:57:21,/TheBobbyM/status/1300945875691171840,You say double standards we say false equivale...,en,True
3353,trump,Raul Anorve,2020-09-01 23:57:21,/AnorveRaul/status/1300945874500030465,Trump has slipped among key groups that backed...,en,False


In [7]:
df[df.search == "biden"]

,search,author,time,url,text,lang,reply
0,biden,KS MANN,2020-09-01 23:59:59,/KSMANN/status/1300946538986909696,Ari Fleischer warns Biden handlers 'severely' ...,en,False
1,biden,ᴮᴱ⁷BANGᴛᴀɴɴɪᴇ✜𖧵 | Via’s Only,2020-09-01 23:59:59,/chicky_nug1bts1/status/1300946538840109056,Love trump is fucking mediocre but Biden ain’t...,en,True
2,biden,Neighbors & Citizens,2020-09-01 23:59:59,/CoalitionofNC/status/1300946538089373697,There guy is President now. They are touring T...,en,False
3,biden,People Push,2020-09-01 23:59:58,/wepeoplepush/status/1300946533400162305,Funny how Biden is just a citizen now and THIS...,en,True
4,biden,George Kneisser,2020-09-01 23:59:58,/GeorgeKneisser/status/1300946532384960512,And Biden is supposedly the one with dementia.,en,False
...,...,...,...,...,...,...,...
1805,biden,Just call me Pj [Blue heart]⚡[Splashing sweat ...,2020-09-01 23:48:51,/DalrymplePamala/status/1300943736956952577,America has been saying @realDonaldTrump was ...,en,False
1806,biden,Gina,2020-09-01 23:48:51,/geenahwilcox/status/1300943735413338112,They’re trying to hide the fact the Biden had ...,en,True
1807,biden,Rue de l’avenir,2020-09-01 23:48:50,/jasonmantis/status/1300943732997488640,I'll believe you when you show me the Biden cl...,en,True
1808,biden,Alejandro,2020-09-01 23:48:50,/Alexpanther56/status/1300943730908815360,Si gracias a los gobiernos que lo antecedierón...,es,True
